In [1]:
from HtmlCss import HtmlCSS
import ipywidgets as widgets
from IPython.display import display
import requests
from GlobalStatsPlots import GlobalStatsPlots


employee_name_widget = widgets.Label(value='Bienvenido al Sistema Analítico Hotelero')
section_titles = [("login", "Login Section"),
              ("local_statistics", "Local Statistics"),
              ("global_statistics", "Global Statistics"),
              ("create_records", "Create Records"),
              ("update_delete_records", "Update Delete Records")]
HtmlCSS().create_navBar(section_titles)
display(employee_name_widget)


global_statistic_widget = widgets.Output()
def mostrar_graficos_most_reservation():
    with global_statistic_widget:
        GlobalStatsPlots().mostrar_graficos_most_reservation(session=session)
        
def graficar_capacidad_hoteles():
    with global_statistic_widget:
        GlobalStatsPlots().graficar_capacidad_hoteles(session=session)
        


# El widget Label para mostrar mensajes
mensaje_label_global_stats = widgets.Label()
# Aquí está tu función actualizada para verificar el acceso y mostrar gráficos si es necesario
def actualizar_interfaz_global_stats(cookies):
    if not cookies:
        mensaje_label_global_stats.value = "Necesitas hacer log in para ver la información en este área."
        # Asegurarse de que el área de gráficos esté vacía o escondida
        with global_statistic_widget:
            global_statistic_widget.clear_output()
    elif cookies.get('position', '') == 'Administrator':
        mensaje_label_global_stats.value = "Gráficos Globales"
        # Mostrar los gráficos porque el usuario es un administrador
        mostrar_graficos_most_reservation()
        graficar_capacidad_hoteles()
    else:
        mensaje_label_global_stats.value = "No tienes permiso para ver esta sección."
        # Asegurarse de que el área de gráficos esté vacía o escondida
        with global_statistic_widget:
            global_statistic_widget.clear_output()
            
            
            
local_statistic_widget = widgets.Output()
mensaje_label_local_statistic = widgets.Label()
dropdown_hotels = widgets.Dropdown(description='Hotel:')
button_show_stats = widgets.Button(description='Mostrar Estadísticas')

def actualizar_interfaz_local_stats(cookies):
    if not cookies:
        mensaje_label_local_statistic.value = "Necesitas hacer log in para ver la información en este área."
        # Asegurarse de que el área de gráficos esté vacía o escondida
        with local_statistic_widget:
            local_statistic_widget.clear_output()
    else:
        mensaje_label_local_statistic.value = "Estadisticas locales:"
        url_accessible_hotels = f'http://127.0.0.1:5000/employee/{session.cookies.get_dict()['eid']}/accessible-hotels'
        
        # Realizar la petición al endpoint
        response = session.get(url_accessible_hotels)
        # Depuración de la respuesta
        #print(f"Respuesta: {response.status_code}, Cuerpo: {response.text}")
        if response.status_code == 200:
            hotels_data = response.json()
            # Crear un dropdown para seleccionar el hotel
            dropdown_hotels.options = [(hotel['hname'], hotel['hid']) for hotel in hotels_data]

            # Botón para confirmar la selección y mostrar estadísticas
            
            def on_button_clicked(b):
                with local_statistic_widget:
                    local_statistic_widget.clear_output()

                    # Aquí debes añadir la lógica para obtener y mostrar las estadísticas del hotel seleccionado
                    # Puedes usar otro endpoint y pasar el hid seleccionado para obtener las estadísticas
                    print(f"Mostrando estadísticas para el hotel {dropdown_hotels.label}")
            
            button_show_stats.on_click(on_button_clicked)
        else:
            print("Error al obtener la lista de hoteles accesibles.")
            
            
            


submit_login = widgets.Output()
# Widget para mostrar la información almacenada en las cookies



username_input = widgets.Text(
    value='',
    placeholder='Ingrese su nombre de usuario',
    description='Usuario:',
    disabled=False
)

password_input = widgets.Password(
    placeholder='Ingrese su contraseña',
    description='Contraseña:',
    disabled=False
)

login_button = widgets.Button(
    description='Iniciar sesión',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Enviar',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

session = requests.Session()

@submit_login.capture(clear_output=True)
def on_login_button_clicked(b):
    # Aquí usarías `requests` para enviar los datos al servidor
    url = 'http://127.0.0.1:5000/login/in/frontend'
    data = {
        'username': username_input.value,
        'password': password_input.value
    }
    response = session.post(url, json=data, verify=False)  # `session` debería ser una instancia de `requests.Session()` ya creada
    
    # Ahora, maneja la respuesta
    if response.status_code == 200:
        print("Login exitoso")
        # Actualizar la información de las cookies
        employee_name_widget.value = "Bienvenido " + str(session.cookies.get_dict()['fname'] + " " + session.cookies.get_dict()['lname'] + " ") + "al Sistema Analítico Hotelero"
        # Aquí podrías actualizar el notebook o hacer algo con la respuesta
        actualizar_interfaz_global_stats(session.cookies.get_dict())
        actualizar_interfaz_local_stats(session.cookies.get_dict())
        username_input.close()
        password_input.close()
        login_button.close()
    else:
        print("Usuario o contaseña incorrecta")
        
login_button.on_click(on_login_button_clicked)



#Seccion para hacer log in

HtmlCSS().create_styled_section(section_titles[0][0], section_titles[0][1])
display(username_input, password_input, login_button, submit_login)

#seccion para mostrar los local stats
HtmlCSS().create_styled_section(section_titles[1][0], section_titles[1][1])
display(mensaje_label_local_statistic, dropdown_hotels, button_show_stats, local_statistic_widget)
actualizar_interfaz_local_stats(session.cookies.get_dict())


#seccion para mostrar los global stats
HtmlCSS().create_styled_section(section_titles[2][0], section_titles[2][1])
display(mensaje_label_global_stats, global_statistic_widget)
actualizar_interfaz_global_stats(session.cookies.get_dict())

#seccion para crear nuevos recors
HtmlCSS().create_styled_section(section_titles[3][0], section_titles[3][1])

#sexxion para borrar y actualizar nuevos records
HtmlCSS().create_styled_section(section_titles[4][0], section_titles[4][1])



HTML(value='\n        <style>\n            #navbar {\n              background-color: #FF5733; /* Color de fon…

Label(value='Bienvenido al Sistema Analítico Hotelero')

Text(value='', description='Usuario:', placeholder='Ingrese su nombre de usuario')

Password(description='Contraseña:', placeholder='Ingrese su contraseña')

Button(description='Iniciar sesión', icon='check', style=ButtonStyle(), tooltip='Enviar')

...

HTML(value='\n        <style>\n        #global_statistics {\n          color: white; /* Cambia el color del te…

Label(value='')

Output()

HTML(value='\n        <style>\n        #create_records {\n          color: white; /* Cambia el color del texto…

HTML(value='\n        <style>\n        #update_delete_records {\n          color: white; /* Cambia el color de…